In [2]:
from zapbench import constants
from zapbench import data_utils
from zapbench.ts_forecasting import data_source


In [3]:
sources = []
num_timesteps_context = 4
# Iterate over all training conditions (excludes 'taxis'), and create
# data sources.
for condition_id in constants.CONDITIONS_TRAIN:
  config = data_source.TensorStoreTimeSeriesConfig(
      input_spec=data_utils.adjust_spec_for_condition_and_split(
          condition=condition_id,
          split='train',
          spec=data_utils.get_spec('240930_traces'),
          num_timesteps_context=num_timesteps_context),
      timesteps_input=num_timesteps_context,
      timesteps_output=constants.PREDICTION_WINDOW_LENGTH,
  )
  sources.append(data_source.TensorStoreTimeSeries(config, prefetch=True))

# Concatenate into a single source.
train_source = data_source.ConcatenatedTensorStoreTimeSeries(*sources)

f'{len(train_source)=}'


E0514 16:26:48.920815 163167315 google_auth_provider.cc:188] Could not find the credentials file in the standard gcloud location [/Users/ayaam/.config/gcloud/application_default_credentials.json]. You may specify a credentials file using $GOOGLE_APPLICATION_CREDENTIALS, or to use Google application default credentials, run: gcloud auth application-default login


'len(source)=4772'

In [6]:
sources = []
for condition_id in constants.CONDITIONS_TRAIN:
    config = data_source.TensorStoreTimeSeriesConfig(
        input_spec=data_utils.adjust_spec_for_condition_and_split(
            condition=condition_id,
            split='val',
            spec=data_utils.get_spec('240930_traces'),
            num_timesteps_context=num_timesteps_context
        ),
        timesteps_input = num_timesteps_context,
        timesteps_output=constants.PREDICTION_WINDOW_LENGTH,
    )
    sources.append(data_source.TensorStoreTimeSeries(config, prefetch=True))

val_source = data_source.ConcatenatedTensorStoreTimeSeries(*sources)
f'{len(val_source)=}'

'len(val_source)=469'

In [21]:
import grain.python as grain
batch_size = 8
num_epochs = 1
shuffle = True

train_index_sampler = grain.IndexSampler(
    num_records=len(train_source),
    num_epochs=num_epochs,
    shard_options=grain.ShardOptions(
        shard_index=0, shard_count=1, drop_remainder=True),
    shuffle=shuffle,
    seed=101
)

train_data_loader = grain.DataLoader(
    data_source=train_source,
    sampler=train_index_sampler,
    operations=[
        grain.Batch(
            batch_size=batch_size, drop_remainder=True)
    ],
    worker_count=0
)

val_index_sampler = grain.IndexSampler(
    num_records=len(val_source),
    num_epochs=num_epochs,
    shard_options=grain.ShardOptions(
        shard_index=0, shard_count=1, drop_remainder=True),
    shuffle=shuffle,
    seed=101
)

val_data_loader = grain.DataLoader(
    data_source=val_source,
    sampler=val_index_sampler,
    operations=[
        grain.Batch(
            batch_size=batch_size, drop_remainder=True
        )],
    worker_count=0
)


In [22]:

loader_dict = {
    'train': train_data_loader,
    'val': val_data_loader,
}
def get_batch(split: str):
    return next(iter(loader_dict[split]))


In [20]:
for iteration, element in enumerate(loader_dict['train']):
    if iteration % 100 == 0:
        print("val")


val
val
val
val
val
val


In [25]:
from src.Transformer.thingy import GPTLanguageModel
import torch
model = GPTLanguageModel()
model.load_state_dict(torch.load('model.pt'))


NameError: name 'vocab_size' is not defined